# Chapter 16: A Deeper Kind of Network

## The Echidna Dataset

In [ ]:
# Load Echidna dataset

import os
import numpy as np


def load(filename):
    data = np.loadtxt(filename, skiprows=1, unpack=True).T
    np.random.seed(12345)
    np.random.shuffle(data)
    x_raw = data[:, 0:2]
    x_min = x_raw.min(axis=0)
    x_max = x_raw.max(axis=0)
    # Rescale data between -0.5 and 0.5
    x = (x_raw - x_min) / (x_max - x_min) - 0.5
    y = data[:, 2].astype(int).reshape(-1, 1)
    return (x, y)


X, Y = load('./echidna.txt')
X_train, X_validation, X_test = np.split(X, 3)
Y_train, Y_validation, Y_test = np.split(Y, 3)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Plot the Echidna dataset---all of it, and then split into training,
# validation and test sets.

def plot_data_by_label(input_variables, labels, label_selector, symbol):
    points = input_variables[(labels == label_selector).flatten()]
    plt.plot(points[:, 0], points[:, 1], symbol, markersize=4)


def plot_data(title, x, y):
    RANGE = 0.55
    plt.xlim(-RANGE, RANGE)
    plt.ylim(-RANGE, RANGE)
    plot_data_by_label(x, y, 0, 'bs')
    plot_data_by_label(x, y, 1, 'g^')
    plt.gca().axes.xaxis.set_ticklabels([])
    plt.gca().axes.yaxis.set_ticklabels([])
    plt.title(title)
    plt.ion()
    plt.show()


plot_data("All data", X, Y)
plot_data("Training set", X_train, Y_train)
plot_data("Validation set", X_validation, Y_validation)
plot_data("Test set", X_test, Y_test)

## Building a Neural Network with Keras

In [ ]:
# Plot the decision boundary of a Keras model over a bi-dimensional dataset.

from matplotlib.colors import ListedColormap


def plot_boundary(model, points):
    # Generate a grid of points over the data
    RANGE = 0.55
    x_mesh = np.arange(-RANGE, RANGE, 0.001)
    y_mesh = np.arange(-RANGE, RANGE, 0.001)
    grid_x, grid_y = np.meshgrid(x_mesh, y_mesh)
    grid = np.c_[grid_x.ravel(), grid_y.ravel()]
    # Classify points in the grid
    classifications = model.predict(grid).argmax(axis=1)
    classifications_grid = classifications.reshape(grid_x.shape)
    # Trace the decision boundary
    BLUE_AND_GREEN = ListedColormap(['#BBBBFF', '#BBFFBB'])
    plt.contourf(grid_x, grid_y, classifications_grid, cmap=BLUE_AND_GREEN)


def plot_data_by_label(input_variables, labels, label_selector, symbol):
    points = input_variables[(labels == label_selector).flatten()]
    plt.plot(points[:, 0], points[:, 1], symbol, markersize=4)


def show_boundary(model, x, y, title="Decision boundary"):
    plot_boundary(model, x)
    plot_data_by_label(x, y, 0, 'bs')
    plot_data_by_label(x, y, 1, 'g^')
    plt.gca().axes.xaxis.set_ticklabels([])
    plt.gca().axes.yaxis.set_ticklabels([])
    plt.title(title)
    plt.ion()
    plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
from keras.utils import to_categorical

Y_train_one_hot_encoded = to_categorical(Y_train)
Y_validation_one_hot_encoded = to_categorical(Y_validation)

shallow_model = Sequential()
shallow_model.add(Dense(100, activation='sigmoid'))
shallow_model.add(Dense(2, activation='softmax'))

shallow_model.compile(loss='categorical_crossentropy',
                      optimizer=RMSprop(lr=0.001),
                      metrics=['accuracy'])

shallow_model.fit(X_train, Y_train_one_hot_encoded,
                  validation_data=(X_validation, Y_validation_one_hot_encoded),
                  epochs=30000, batch_size=25)

In [ ]:
show_boundary(shallow_model, X_train, Y_train)

## Making it Deep

In [ ]:
deep_model = Sequential()
deep_model.add(Dense(100, activation='sigmoid'))
deep_model.add(Dense(30, activation='sigmoid'))
deep_model.add(Dense(2, activation='softmax'))

deep_model.compile(loss='categorical_crossentropy',
                   optimizer=RMSprop(lr=0.001),
                   metrics=['accuracy'])

deep_model.fit(X_train, Y_train_one_hot_encoded,
               validation_data=(X_validation, Y_validation_one_hot_encoded),
               epochs=30000, batch_size=25)

In [ ]:
show_boundary(deep_model, X_train, Y_train)